In [1]:
import import_ipynb
# 원하는 모델로 바꾸어 사용하기
from vision_model_final_sol import Mini_Xception

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import torch
import torchvision.transforms.transforms as transforms

importing Jupyter notebook from vision_model_final_sol.ipynb


In [2]:
# (주의) OpenCV는 Colab에서 제대로 실행되지 않음.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 사용할 모델 선언하기
mini_xception = Mini_Xception().to(device)

# train이 아닌, evaluation 과정
mini_xception.eval()

# 기존에 학습한 모델 불러오기. XX에 epoch 번호 작성.
checkpoint = torch.load('checkpoint/model_weights/weights_epoch_2.pth.tar', map_location=device)
mini_xception.load_state_dict(checkpoint['mini_xception'])

# Face detection을 위한 CascadeClassifier 모델 불러오기
path = "haarcascade_frontalface_default.xml"
face_detector = cv2.CascadeClassifier(path)

In [4]:
def sort_vec(vec):
    """
    7차원 감정 벡터의 순서를 일치시키기 위한 함수
        
    매개변수 (Parameters)
    ----------
    vec : numpy.ndarray, shape-(7,)
        vision 모델의 출력값인 7차원 벡터.
        fer2013 dataset의 emotion label의 순서대로 구한 감정 벡터. 
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape-(7,)
        NLP dataset의 emotion index의 순서대로 재정렬한 감정 벡터
    
    참고사항
    -------
    (fer2013 dataset)
        0: 'Angry',
        1: 'Disgust', 
        2: 'Fear', 
        3: 'Happy', 
        4: 'Sad', 
        5: 'Surprise', 
        6: 'Neutral'
        
    (NLP dataset)
        0: 'Fear',
        1: 'Surprise', 
        2: 'Angry', 
        3: 'Sad', 
        4: 'Neutral', 
        5: 'Happy', 
        6: 'Disgust'  
    """
    
    vec2 = [vec[2],vec[5],vec[0],vec[4],vec[6],vec[3],vec[1]]
    
    return np.array(vec2)

In [5]:
def get_label_emotion(label):
    """
    label 값에 대응되는 감정의 이름 문자열을 구하기 위한 함수
        
    매개변수 (Parameters)
    ----------
    label : int
        emotion label 번호
        
    반환 값 (Returns)
    -------
    String
        label 번호에 대응되는 감정의 이름 문자열
    
    참고사항
    -------
    (NLP dataset)
        0: 'Fear',
        1: 'Surprise', 
        2: 'Angry', 
        3: 'Sad', 
        4: 'Neutral', 
        5: 'Happy', 
        6: 'Disgust'      
    """
    
    label_emotion_map = {
        0: 'Fear',
        1: 'Surprise', 
        2: 'Angry', 
        3: 'Sad', 
        4: 'Neutral', 
        5: 'Happy', 
        6: 'Disgust'        
    }
    
    return label_emotion_map[label]

In [6]:
def cos_sim(A, B):
    """
    두 벡터 A, B의 코사인 유사도를 구하기 위한 함수
        
    매개변수 (Parameters)
    ----------
    A : numpy.ndarray, shape-(N,)
    B : numpy.ndarray, shape-(N,)
        
    반환 값 (Returns)
    -------
    numpy.float64
        두 벡터 A, B의 코사인 유사도 값      
    """
    
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [7]:
def predict_video(nlp_vec, sentence):
    """
    웹캠을 통해 받아온 실시간 영상 속에서
    1) CascadeClassifier (혹은 다른 모델)을 통해 얼굴을 탐지하고
    2) Mini_Xception (혹은 다른 모델)을 통해 얼굴 표정으로부터 7차원 감정 벡터를 추출하여
    3) sort_vec 함수를 통해 2)에서 구한 감정 벡터의 순서를 재정렬한 후
    4) cos_sim 함수를 이용하여 입력받은 문장에서 추출한 7차원 감정 벡터와의 코사인 유사도를 계산.
    5) OpenCV 라이브러리를 사용하여, 문장과 표정에서 추출한 각 감정, 그리고 표정 연기에 대한 점수(코사인 유사도)를 window 상에 시각화.
    
    매개변수 (Parameters)
    ----------
    nlp_vec : 입력한 문장에서 추출한 7차원 감정 벡터
    sentence : 사용자가 표정 연기 연습의 목적으로 입력한 문장
    """
    
    # OpenCV 실시간 웹캠 영상 불러오기
    cap = cv2.VideoCapture(0)

    # 프레임의 사이즈 계산 (height, width 구하기)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    size = (width, height)
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret:
            # 문장의 최대 확률 감정
            emotion_max = np.argmax(nlp_vec)
            nlp_percentage = np.round(nlp_vec[emotion_max], 2)
            nlp_emotion_label = get_label_emotion(emotion_max)
            
            # 한글 문장 출력을 위한 하단 색 띠
            frame_pil = frame
            frame_pil[height-70 : height, 0 : width] = (50, 100, 50)
            
            # 한글 문장 출력을 위한 PIL 라이브러리 사용
            frame_pil = Image.fromarray(cv2.cvtColor(frame_pil, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(frame_pil)
            
            msg1 = "emotion : " + nlp_emotion_label + " (" + str(nlp_percentage) + ")"
            msg2 = "\"" + sentence + "\""
            font = ImageFont.truetype("Hancom Gothic Regular.ttf", 20)
            w1, h1 = font.getsize(msg1)
            w2, h2 = font.getsize(msg2)
            draw.text(((width-w1)/2, height-h1-40), msg1, font = font, fill = (255, 255, 255))
            draw.text(((width-w2)/2, height-h2-10), msg2, font = font, fill = (255, 255, 255))
            
            # 한글 문장이 출력되어 있는 프레임
            frame_GUI = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
            
            # faces : 얼굴 탐지 결과 얻어진, face(x,y,w,h)로 이루어진 sequential data 
            faces = face_detector.detectMultiScale(frame)
            
            
            for face in faces:
                (x,y,w,h) = face
            
                # 웹캠에서 인식한 얼굴을 모델에 넣어주기 위한 전처리
                '''
                전처리 후, input_face에 저장
                 1) face의 좌표에 따라 얼굴 부분 프레임만 잘라내기
                 2) BGR2GRAY로 흑백 변환하기
                 3) (48,48)로 resize
                 4) 히스토그램 평활화 적용
                 5) Tensor로 바꾸고 device에 저장
                 6) (1,48,48)로 차원 증가
                '''
                input_face = frame[y:y+h, x:x+w]
                input_face = cv2.cvtColor(input_face, cv2.COLOR_BGR2GRAY)
                input_face = cv2.resize(input_face, (48,48))
                input_face = cv2.equalizeHist(input_face)
                input_face = transforms.ToTensor()(input_face).to(device)
                input_face = torch.unsqueeze(input_face, 0)

                
                with torch.no_grad():
                    # 모델 출력값의 shape : [1, 7, 1, 1]
                    emotion_vec = mini_xception(input_face).squeeze()
                    
                    # 7차원 감정 확률 벡터
                    softmax = torch.nn.Softmax()
                    vision_vec = softmax(emotion_vec)
                    vision_vec = vision_vec.reshape(-1,1).cpu().detach().numpy()
                    
                    # 코사인 유사도 점수
                    vision_vec = sort_vec(vision_vec)
                    similarity = cos_sim(nlp_vec,vision_vec)
                    
                    # GUI 상에서 출력할 정보
                    '''
                     1) 한글 문장과 최대 확률 감정, 그 확률 (이미 PIL 라이브러리로 해결)
                     2) 코사인 유사도 점수, score
                     3) 표정의 최대 확률 감정과 그 확률
                    '''
                    score = np.round(similarity * 100, 2)
                    
                    emotion_max = np.argmax(vision_vec)
                    vision_percentage = np.round(vision_vec[emotion_max], 2)
                    vision_emotion_label = get_label_emotion(emotion_max)
                    
                    # 얼굴 표정 주변의 정보 출력을 위한 OpenCV 라이브러리 사용
                    cv2.rectangle(frame_GUI, (x, y), (x + w, y + h), (255,0,0), 3)
                    frame_GUI[y - 60 : y, x : x + w] = (50,50,50)
                    frame_GUI[y + h - 60 : y, x : x + w] = (70,30,30)
                    
                    cv2.putText(frame_GUI, "Score: " + str(score), (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,200,200))
                    cv2.putText(frame_GUI, vision_emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,200))
                    cv2.putText(frame_GUI, str(vision_percentage), (x + w - 50, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,0))            
            
            cv2.imshow("Video", frame_GUI)
            # 탈출 조건 : esc ( OxFF==27 )
            if cv2.waitKey(1) & 0xff == 27:
                break
            
        else:
            break
    
    # 종료
    cap.release()
    cv2.destroyAllWindows()